- Maak een VIEW voor fd_resultaten_odds_view met geconformeerde teamnamen en wedkantoren
- Maak een VIEW voor oddapi_odds_VIEW, met geconformeerd teamnamen en wedkantoren

- UNION de bovenstaande VIEWS en haal wedstrijdsleutel op
- JOIN met f_odds en selecteer de odds die nog ontbreken in f_odds
- In dat geval, INSERT in f_odds

- UPDATE datum update

In [2]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2
import datetime

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

# Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()


# update upate_tabel
psy_cursor.execute("""
INSERT INTO dwa.f_odds 
SELECT
odd_conf.s_wedstrijd
,odd_conf.wedkantoor
,odd_conf.datum
,odd_conf.odd_type
,cast(odd_conf.h_odd_price as decimal)
FROM (
	SELECT 
	s_wedstrijd
	,wedkantoor
	,datum 
	,'homeodd' as odd_type
	,h_odd_price 
	FROM (
		SELECT * FROM dsa.fd_stats_odds_view fsov
		UNION
		SELECT * FROM dsa.oddapi_odds_view oov 
		ORDER BY div, datum
		) all_odds
	LEFT OUTER JOIN dwa.d_wedstrijden dw 
		ON all_odds.hometeam = dw.hometeam AND all_odds.awayteam = dw.awayteam
	) odd_conf
LEFT OUTER JOIN dwa.f_odds fo 
	ON fo.s_wedstrijd = odd_conf.s_wedstrijd AND fo.wedkantoor = odd_conf.wedkantoor AND fo.odd_datum = odd_conf.datum
WHERE fo.s_wedstrijd IS NULL AND fo.wedkantoor IS NULL AND fo.odd_datum IS NULL;       

UPDATE dsa.update
SET update_datum = current_date
WHERE tabelnaam = 'f_odds';                      
""")

# Commit de transactie en sluit de verbinding
sqal_connection.commit()
psy_connection.commit()
psy_cursor.close()
sqal_connection.close()
psy_connection.close() 